# Kosovo Population Animation

## Preprocessing

In [73]:
import pandas as pd

df_mun = pd.read_csv("../data/kosovo-municipalities.csv")
df_pop = pd.read_csv("../data/2012-2018_population.csv", encoding = "ISO-8859-1", sep=";")

In [74]:
# Melt dataframe
df_pop.columns = ["komuna", "31-12-2012", "31-12-2013", "31-12-2014", "31-12-2015", "31-12-2016", "31-12-2017", "31-12-2018"]
df_pop = df_pop.melt(id_vars=["komuna"], var_name="year", value_name="population")

In [75]:
df_pop["year"] = df_pop["year"].apply(lambda x: x[-4:])

In [76]:
# remove special characters
df_pop.replace("ë", "e", inplace=True, regex=True)
df_pop.replace("ç", "c", inplace=True, regex=True)

### Municipalities data

In [193]:
from pathlib import Path

df_all = pd.DataFrame(columns=["komuna", "year", "population"])

pathlist = Path("../data/municipalities-data/").glob('*.csv')
for path in pathlist:
    komuna = str(path)[str(path).find("_")+1:(str(path).find(".csv"))]
    df = pd.read_csv(str(path), header=1, encoding = "ISO-8859-1", sep=";")
    df.columns = ["komuna", "1948", "1953", "1961", "1971", "1981", "1991", "2011"]
    df = df.melt(id_vars=["komuna"], var_name="year", value_name="population")
    df = pd.concat([df, df_pop.loc[df_pop["komuna"]==komuna]], ignore_index=True)
    df["komuna"] = komuna
    # TODO(Interpolate the years in between)
    df.set_index("year", inplace=True)
    df_temp = pd.DataFrame(index=[str(i).zfill(2) for i in range(1948,2019)], columns=["komuna", "population"])
    df_temp.index.rename('year', inplace=True)
    df_temp.update(df)
    df_temp.reset_index(inplace=True)
    df_temp["komuna"] = komuna
    df_temp["year"] = pd.to_numeric(df_temp["year"])
    df_temp["population"] = pd.to_numeric(df_temp["population"])
    df_temp["population"] = df_temp["population"].interpolate()
    df_all = pd.concat([df_all, df_temp], ignore_index=True)

### Add coordinates

In [195]:
# Merge with coordinates after you have everything
df = pd.merge(df_all, df_mun[["X", "Y", "nam"]], how="left", left_on = "komuna", right_on = "nam")

In [198]:
df.drop("nam",inplace=True,axis=1)

In [201]:
df.to_csv("../data/population.csv")

## Shapefile

In [103]:
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

In [104]:
sns.set(style="whitegrid", palette="pastel", color_codes=True)
sns.mpl.rc("figure", figsize=(10,6))

In [105]:
%matplotlib inline

In [106]:
import shapefile as shp  # Requires the pyshp package
import matplotlib.pyplot as plt

sf = shp.Reader("../data/kosovo-shapefile/XK_EA_2018.shp")

## Plotting

In [ ]:
# A lot easier with gganimate in R